Script Práctica 1 Parte 2 (MLP)

Redes de Neuronas Artificiales, curso 2022-2023

Universidad Carlos III de Madrid

# Importar TensorFlow, ciertos módulos de Keras y las librerías numpy y pandas.

In [3]:
""" Regression problem: MLP con Keras """
import tensorflow as tf
import numpy as np
import pandas as pd
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense

Para trabajar con un repositorio almacenado en drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

#!ls

#%cd "/content/drive/MyDrive/RN/Practica1/parte1"

#!ls

ModuleNotFoundError: No module named 'google.colab'

# Cargar los datos

* Hay que ajustar el path a los csv de datos 
* Cambiar parámetros correspondientes si el delimitador no es una coma, o si el archivo contiene cabeceras
* Pueden cargarse numpy arrays o dataframes, en este caso se van a utilizar arrays

In [ ]:
# CARGAR DATOS compactiv
train_set = pd.read_csv('/content/sample_data/Train.csv', header='infer', delimiter=',')
valid_set = pd.read_csv('/content/sample_data/Valid.csv', header='infer', delimiter=',')
test_set = pd.read_csv('/content/sample_data/Test.csv', header='infer', delimiter=',')
# SELECCION DE LA SALIDA. Num de columna del target.
y_train = np.array(train_set.iloc[:,-1:])
X_train = np.array(train_set.iloc[: , :-1])
y_valid = np.array(valid_set.iloc[:,-1:])
X_valid = np.array(valid_set.iloc[: , :-1])
y_test = np.array(test_set.iloc[:,-1:])
X_test = np.array(test_set.iloc[: , :-1])
num_train_samples=len(y_train)

# mediante prints es posible COMPROBAR DIMENSIONES DE LOS DATOS para comprobar que todo se haya cargado correctamente
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print('X_valid: ',X_valid.shape)
print('y_valid: ',y_valid.shape)
print('X_test: ',X_test.shape)
print('y_test: ',y_test.shape)

: 

# Ejemplos de modelos: 
* PM con relu 
* PM con Sigmoide
* Adaline (una sola neurona lineal) usando keras

In [ ]:
# Definir forma de la entrada de la red
input_shape=(X_train.shape [1],) # utilizamos los datos de entrenamiento para definir la tupla , en este caso sera (21,)

: 

In [ ]:
def create_PM_sigmoid(num_hidden_neurons = 50):#se pone un valor por defecto a num_hidden_neurons, pero puede llamarse con otro valor
  #1 capa oculta y 1 neurona de salida con sigmoide
  model = Sequential() # generamos el modelo dandole una forma secuencial: 
  model.add(Dense(num_hidden_neurons, input_shape=input_shape, activation='sigmoid')) # capa con x=num_hidden_neurons neuronas activadas con sigmoide
  model.add(Dense(1,activation='sigmoid')) # capa con x=1 neurona (solo una neurona de salida) activada con sigmoide 
  return model
def create_PM_relu(num_hidden_neurons = 50):
#1 capa oculta con relu y 1 neurona de salida con sigmoide
  model = Sequential()
  model.add(Dense(num_hidden_neurons, input_shape=input_shape, activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  return model
#modelo lineal, solo para comparar con el programa desarrollado.
def create_lineal(): # Una sola neurona lineal (Adaline)
  model = Sequential()
  model.add(Dense(1, input_shape=input_shape, activation='linear'))
  return model

: 

# Definir el modelo de red de neuronas. Utilizando las funciones antes definidas


In [ ]:
#Seleccionar el modelo llamando a la función correspondiente
#model=create_PM_relu(100) # 1 capa relu con 100 neuronas y salida sigmoid
model=create_PM_sigmoid(20) # 1 capa sigmoid con 20 neuronas y salida sigmoid
#model=create_lineal() # 1 capa salida LINEAL (ADALINE)
model.summary() #visualizar la estructura del modelo

: 

# Compilar el modelo y entrenarlo

In [ ]:
# CONFIGURAR MODELO Y ENTRENAMIENTO
lr = 0.2 # razon de aprendizaje
epochs = 300 # numero de ciclos que se quiere realizar en el entrenamiento
batch_size=32 # ¡¡¡¡¡¡ no cambiar este valor !!!!!!!!!!!!!! 
# Para poder ver la curva de validación hay que poner validation_freq=1. Tarda más
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0), metrics=['mse'] )
historico = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(X_valid,y_valid),
shuffle=False, validation_freq=1)


: 

# Plots de evolución del error

In [ ]:
from matplotlib import pyplot as plt
plt.plot(historico.history['loss'])
plt.plot(historico.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

: 

#  Evaluar el modelo con el conjunto de test

In [ ]:
#Evaluar el modelo con el fich de test
results_test = model.evaluate(X_test, y_test)
results_test[0]
#resultado es una lista con los valores de loss y las métricas elegidas. 
#En este caso son loss y mse que coinciden.

: 

# Obtener la predicción para el conjunto de test.

In [ ]:
# predicciones en test
test_pred = model.predict(X_test)
#crear un ndarray: columna 1 predicciones, columna 2 target
comp = np.append(test_pred, y_test, axis = 1)
comp_df = pd.DataFrame(comp, columns=['prediccion','target'])
print(comp_df.iloc[:10,:])
#mostrar 10 primeras filas

: 

# Guardar los resultados de entrenamiento en ficheros

In [ ]:
# GUARDAR RESULTADOS EN FICHEROS
# evolución del entrenamiento 
# son los datos que se usan para construir los plots
# En este caso, la variable 'historico' contiene los datos del último entrenamiento realizado
np.savetxt('historicoTrainLoss.txt',historico.history['loss'])
np.savetxt('historicoValLoss.txt',historico.history['val_loss'])

: 

# Guardar los resultados finales en ficheros

In [ ]:
#guarda el modelo completo
model.save('modelo.h5')
#guarda solo pesos
model.save_weights('pesos.h5')
#cargar el modelo guardado
#from keras.models import load_model
#new_model = load_model('modelo.h5')

: 